# Test Environment

This notebook serves as an environment to test various scripts and bits of code.  Nothing here should be dependend on.

In [1]:
# set the project path
%cd ~/projects/drug-pricing

/Users/cccdenhart/projects/drug-pricing


In [2]:
%load_ext autoreload

In [27]:
%autoreload 2
import pandas as pd
from tqdm import tqdm

from src.utils import connect_to_mongo, get_nlp
from src.schema import User, Post, SubmissionPost, CommentPost, Location
from src.models.v1.__init__ import load_locations, V1, DENYLIST, DenylistFilter, LocationFilter, FrequencyRanker

In [59]:
pd.set_option('display.max_rows', 1000)

In [28]:
connect_to_mongo()

In [47]:
from src.scripts.aggregate_locations import main

In [95]:
main()

Reading data .....
Transforming data .....
Wrote grouped locations to data/locations/grouped-locations.csv .....


In [96]:
df = pd.read_csv("data/locations/grouped-locations.csv")

In [75]:
neigh = pd.read_csv("data/locations/neighborhoods.csv")

In [61]:
cs = pd.read_csv("data/locations/cities-states.csv", sep="|")

In [29]:
%%prun
locations = load_locations("data/locations/grouped-locations.csv")
filters = {
    DenylistFilter(DENYLIST),
    LocationFilter(locations)
}
rankers = {FrequencyRanker()}
nlp = get_nlp()

# instantiate model
model = V1(filters, rankers, locations, nlp)

         205899787 function calls (202835314 primitive calls) in 128.358 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   381121    8.708    0.000   23.943    0.000 document.py:59(__init__)
 28705139    8.675    0.000   14.294    0.000 {built-in method builtins.isinstance}
  5335694    5.541    0.000    9.261    0.000 document.py:166(__setattr__)
  1905605    5.031    0.000   28.200    0.000 base.py:4702(get_value)
 12958660    4.058    0.000    5.619    0.000 generic.py:7(_check)
   381129    3.444    0.000    5.510    0.000 {pandas._libs.lib.infer_dtype}
21351091/21351007    3.319    0.000    6.061    0.000 {built-in method builtins.getattr}
  1905605    3.153    0.000   32.500    0.000 series.py:1068(__getitem__)
  3811215    2.989    0.000    7.883    0.000 {pandas._libs.lib.values_from_object}
381124/381123    2.776    0.000   62.257    0.000 series.py:197(__init__)
  1905605    2.383    0.000    2.383    0.000 {meth

In [30]:
use_top = True
if use_top:
    # get the users with the most posts
    pipeline = [{"$sortByCount": "$user"}, {"$limit": 6}]
    res = Post.objects().aggregate(pipeline)
    ids = [str(r["_id"]) for r in res if r["_id"]]
    users = User.objects(id__in=ids)
else:
    usernames = pd.read_csv("data/rand_user_200.csv", squeeze=True).tolist()
    users = User.objects(username__in=usernames).all()

In [31]:
u = list(users)[0]

In [32]:
entscores = model.score_entities(u)

In [ ]:
%prun locscores = model.score_locations(u)

In [18]:
l = Location(neighborhood="roxbury", city="boston", county="boston", state="ma")

In [20]:
locs = l.match_entity("boston")

In [25]:
for l in locs:
    print(l.to_mongo())

SON([('city', 'boston')])
SON([('county', 'boston')])
SON([('city', 'boston'), ('county', 'boston')])


In [13]:
%debug

> /Users/cccdenhart/projects/drug-pricing/src/schema.py(46)<listcomp>()
     44 
     45         possible_locations = [Location(**{k:self_map[k] for k in self_map.keys() & field_combo})
---> 46                               for field_combo in field_combinations]
     47 
     48         return possible_locations



ipdb>  field_combinations


*** NameError: name 'field_combinations' is not defined


ipdb>  quit
